# Water molecule

This notebook demonstrates how to combine `pyscf` with the DMRG implementation in `chemtensor` for approximating the ground state of the water molecule.

In [1]:
import numpy as np

# For simplicty, we locate the compiled module library in the build folder.
# In the future, it will be installed as part of a Python package.
import sys
sys.path.append("../../build/")
import chemtensor

# pyscf (https://pyscf.org/) can define a molecular basis, compute overlap integrals and run other computational methods for comparison.
import pyscf

### Define the molecule and perform reference calculations

In [2]:
h2o_atoms = [
    ["O", ( 0.,   0.,   0.)],
    ["H", ( 0.75, 0.47, 0.)],
    ["H", (-0.75, 0.47, 0.)],
]
mol = pyscf.M(atom=h2o_atoms, basis="sto-3g")

In [3]:
# run Hartree-Fock
hf = mol.HF().run()

converged SCF energy = -74.9307084821


In [4]:

# run coupled-cluster with single and double excitations (CCSD), for comparison
ccsd = pyscf.cc.CCSD(hf).run()
ccsd.e_tot

E(CCSD) = -74.97016403895398  E_corr = -0.03945555685402184


-74.97016403895398

In [5]:
# run full configuration interaction (FCI)
fcisolver = pyscf.fci.FCI(hf)
en_fci, _ = fcisolver.kernel()
en_fci

-74.9702726895936

### Electron overlap integrals

In [6]:
def spatial_to_spin_overlap_integrals(h1, h2):
    """
    Enlarge the single- and two-particle electron overlap integral tensors
    from an orbital basis without spin to a spin-orbital basis.
    """
    h1 = np.asarray(h1)
    h2 = np.asarray(h2)

    n = h1.shape[0]
    assert h1.shape == (n, n)
    assert h2.shape == (n, n, n, n)

    # single-particle integrals
    h1_so = np.kron(np.eye(2), h1)

    # two-particle integrals
    tmp = np.zeros((2*n, 2*n, n, n))
    for i in range(n):
        for j in range(n):
            tmp[:, :, i, j] = np.kron(np.eye(2), h2[:,:, i, j])
    h2_so = np.zeros((2*n, 2*n, 2*n, 2*n))
    for i in range(2*n):
        for j in range(2*n):
            h2_so[i, j, :, :] = np.kron(np.eye(2), tmp[i, j, :, :])

    return h1_so, h2_so

In [7]:
# overlap integrals in atomic basis
h1_ao = mol.get_hcore()  # == mol.intor("int1e_kin") + mol.intor("int1e_nuc")
eri_ao = mol.intor("int2e")

converged SCF energy = -74.9307084820999


In [8]:
# transform to molecular orbital basis
h1_mo = np.einsum("pi,pq,qj->ij", hf.mo_coeff, h1_ao, hf.mo_coeff)
eri_mo = pyscf.ao2mo.kernel(eri_ao, hf.mo_coeff)
print(h1_mo.shape)
print(eri_mo.shape)

(7, 7)
(7, 7, 7, 7)


In [9]:
# extend to spin-orbitals
h1_so, eri_so = spatial_to_spin_overlap_integrals(h1_mo, eri_mo)
print(h1_so.shape)
print(eri_so.shape)

(14, 14)
(14, 14, 14, 14)


In [10]:
# convert to physicists' convention
tkin = h1_so
vint = np.transpose(eri_so, (0, 2, 1, 3))

### Construct Hamiltonian as MPO and run two-site DMRG

In [11]:
hamiltonian = chemtensor.construct_molecular_hamiltonian_mpo(tkin, vint, optimize=True)
# virtual bond dimensions
hamiltonian.bond_dims

[1, 4, 16, 31, 38, 43, 52, 51, 52, 43, 38, 31, 16, 4, 1]

In [12]:
# number of electrons (determines quantum number sector)
pnum = 10

In [13]:
# run two-site DMRG
psi, en_sweeps, entropy = chemtensor.dmrg(hamiltonian, num_sweeps=6, maxiter_lanczos=25, tol_split=1e-9, qnum_sector=pnum)

### Evaluate results

In [14]:
# virtual bond dimensions of optimized MPS
psi.bond_dims

[1, 2, 4, 7, 10, 14, 19, 19, 20, 16, 11, 7, 4, 2, 1]

In [15]:
# energy after each DMRG sweep
en_sweeps

[-84.88903443027432,
 -84.8890344313337,
 -84.88903443132047,
 -84.88903443131844,
 -84.88903443131815,
 -84.88903443131794]

In [16]:
# add nuclear repulsion energy
en_dmrg = en_sweeps[-1] + hf.energy_nuc()
en_dmrg

-74.97027263097696

In [17]:
# difference to CCSD energy
ccsd.e_tot - en_dmrg

0.00010859202298263426

In [18]:
# difference to FCI energy
en_dmrg - en_fci

5.8616635101316206e-08